In [ ]:
from ngsildclient import Client

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G = nx.DiGraph()
G.add_node("toto")
G.add_node("nimp")
G.add_edge("toto", "nimp")

In [ ]:
plt.tight_layout()
plt.axis("off")
nx.draw_networkx(G, node_color="tab:blue", font_color="whitesmoke")

In [ ]:
import d3graph

In [ ]:
d3graph.d3graph.edges2G(G)

In [ ]:
adjmat = G.adjacency()

In [ ]:
adjmat

In [ ]:
from d3graph import d3graph

In [ ]:
d3 = d3graph()

In [ ]:
d3.graph(adjmat)

In [ ]:
from d3graph import d3graph, vec2adjmat

# Create example network
source = ['node A','node F','node B','node B','node B','node A','node C','node Z']
target = ['node F','node B','node J','node F','node F','node M','node M','node A']
weight = [5.56, 0.5, 0.64, 0.23, 0.9, 3.28, 0.5, 0.45]
# Convert to adjacency matrix
adjmat = vec2adjmat(source, target, weight=weight)

# Initialize
d3 = d3graph()
# Proces adjmat
d3.graph(adjmat)
# Plot
d3.show()


In [ ]:
from ngsildclient import Client
client = Client(port=8026, port_temporal=8027)
root = client.get("A:A1")

In [4]:
from ngsildclient import Client, Entity
a1 = Entity("A", "A1")
b1 = Entity("B", "B1")
c1 = Entity("C", "C1")
d1 = Entity("D", "D1")
d2 = Entity("D", "D2")
a1.rel("hasB", b1)
a1.rel("hasD1", d1)
a1.rel("hasD2", d2)
b1.rel("hasC", c1)
c1.rel("hasA", a1) 

{'@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'id': 'urn:ngsi-ld:C:C1', 'type': 'C', 'hasA': {'type': 'Relationship', 'object': 'urn:ngsi-ld:A:A1'}}

In [5]:
client = Client(port=8026)

Connected to Context Broker at localhost:8026 | vendor=Orion-LD | version=post-v1.1.0


In [6]:
client.upsert([a1, b1, c1, d1, d2])

Entities upserted : 5/5 [1.00]

BatchResult(op='upsert', success=['urn:ngsi-ld:B:B1', 'urn:ngsi-ld:C:C1', 'urn:ngsi-ld:D:D1', 'urn:ngsi-ld:A:A1', 'urn:ngsi-ld:D:D2'], errors=[])

In [ ]:
from ngsildclient import Client, Entity
client = Client(port=8026)
rooms = client.query("RoomObserved")

In [ ]:
from typing import List
def rooms_to_dfdict(rooms: List[Entity]):
    room, pressure, temp = [], [], []
    for r in rooms:
        room.append(r.id.rsplit(":")[-1])
        pressure.append(r["pressure.value"])
        temp.append(r["temperature.value"])
    return {"room": room, "pressure": pressure, "temperature": temp}


In [ ]:
import pandas as pd
d = rooms_to_dfdict(rooms)
df = pd.DataFrame(d)


In [ ]:
import plotly.express as px
fig = px.bar(df, x="room", y="temperature", color="pressure")
fig.show()

In [ ]:
client.close()

In [ ]:
!pip install nbformat

In [8]:
from ngsildclient import Client
client = Client(port=8026)
root = client.get("A:A1")
G = client.network(root)

Connected to Context Broker at localhost:8026 | vendor=Orion-LD | version=post-v1.1.0


In [16]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 3.6 MB/s eta 0:00:0000:0100:01m


In [64]:
import networkx as nx
import plotly.graph_objects as go

# select layout
#pos: dict = nx.spring_layout(G)
pos =pos = nx.shell_layout(G)

# set node positions
node_x = []
node_y = []
node_text = []
node_color = []
node_size = []
current_color: int = 0
maptypecolor: dict[str, int] = {}
for node in G.nodes():
    x, y = G.nodes[node]["pos"]= pos[node]
    node_x.append(x)
    node_y.append(y)
    #print(node)
    type, shortid = node
    node_text.append(shortid)
    color = maptypecolor.get(type)
    if color is None:
        color = current_color
        maptypecolor[type] = color
        current_color += 1
    node_color.append(color)
    node_size.append(15)
node_size[0] = 20
print(node_text)
print(node_color)


['A:A1', 'B:B1', 'C:C1', 'D:D1', 'D:D2']
[0, 1, 2, 3, 3]


In [68]:
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        #showscale=True,
        # colorscale='YlGnBu',
        reversescale=True,
        color=[],
        line_width=2))
node_trace.text = node_text
node_trace.marker.color = node_color
node_trace.marker.size = node_size

In [70]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

In [71]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Entities network graph",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()